In [232]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
import sys
sys.path.append('/localscratch/data/project_one/update/')
from util import find_str
%load_ext autoreload
%autoreload 2#
import requests

from analysts_comments import AnalystsComments
from shorts import ShortsUpdater

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [316]:
ob = AnalystsComments(FileNameListOfCompanies='DowJones.csv',PathData='/localscratch/data/project_one/data/')

In [319]:
ob.web_updater(['MMM'])



Start Stock: MMM
1 None None


KeyboardInterrupt: 

In [274]:

#ob.web_updater(['DAI.DE'])

#x = pd.read_pickle('/localscratch/data/project_one/data/analysts_comments/stocks/DAI.DE.p2')


#print x.head(5)

#x = x.loc[5:]

#x.to_pickle('/localscratch/data/project_one/data/analysts_comments/stocks/DAI.DE.p')

x = pd.read_pickle('/localscratch/data/project_one/data/analysts_comments/stocks/ALV.DE.p')
x.head(5)

# x[['TargetPrize','FormerTargetPrize']]

#plt.scatter(x['TargetPrize'],x['FormerTargetPrize'])
#plt.show()

,Date,TargetPrize,FormerTargetPrize,Category,Analyst,Phrase
0,2018-03-07,194.0,194.0,Neutral,"Merrill Lynch & Co., Inc.",auf 194 Euro belassen.
1,2018-03-01,220.0,220.0,buy,Goldman Sachs Group Inc.,von 220 Euro belassen.
2,2018-02-26,190.0,190.0,Halten,Norddeutsche Landesbank (Nord/LB),von 190 Euro belassen.
3,2018-02-22,195.0,195.0,Hold,Société Générale Group S.A. (SG),von 195 Euro in
4,2018-02-20,176.0,167.0,Neutral,Macquarie Research,von 167 auf 176 Euro angehoben.


In [329]:
def get_phrase(string):
    
    
    string = string.replace(',','.')
       
    ListOfStrings=  string.split(" ")

    indices = [i for i, x in enumerate(ListOfStrings) if x == "US-Dollar" or x =='USD']
    
    print indices
    if len(indices) == 0:
        print("No currency string found ")
        return None
    
    else:
        for ind in indices:                             
            try:
                float(ListOfStrings[ind-1])
                break
                    
            except ValueError:
                if ind == indices[-1]:
                    return None
                
                else:
                    continue
                    

    target = float(ListOfStrings[ind-1])
    formerTarget = None
            
    if ListOfStrings[ind-3].isdigit() is True:
        phrase = " ".join(ListOfStrings[ind-4:ind+2])       
        formerTarget = float(ListOfStrings[ind-3])
    else:
        phrase = " ".join(ListOfStrings[ind-2:ind+2])
        formerTarget = target
    return {"Phrase":phrase,"TargetPrize":target,'FormerTargetPrize':formerTarget}

print string
print "\n"
print get_phrase(string)

London (www.aktiencheck.de) - Shannon O'Callaghan, John Walsh und Christopher Belfiore, Analysten von Nomura Equity Research, stufen die Aktie von 3M Company (ISIN US88579Y1010/ WKN 851745) von "reduce" auf "neutral" hoch. Das Kursziel werde von 88 USD auf 96 USD angehoben. (Analyse vom 09.11.2012) (12.11.2012/ac/a/a)



[35, 38]
{'Phrase': u'von 88 USD auf', 'TargetPrize': 88.0, 'FormerTargetPrize': 88.0}


In [21]:
35 bei intpa = 9

Empty DataFrame
Columns: [Date, Category, Analyst, Phrase, TargetPrize, FormerTargetPrize]
Index: []


In [330]:
page = requests.get("https://www.finanzen.net/analysen/Apple-Analysen@intpagenr_1")

from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')


In [332]:

X = pd.DataFrame(columns=['Date','TargetPrize','FormerTargetPrize','Category','Analyst','Phrase'])
print X


for num,l in enumerate(soup.findAll("table")[1].findAll("tr")):
        
    if num >0:
        #print num
        
        url =  "https://www.finanzen.net"+l.findAll('a')[0].get('href')
        
        tds = l.findAll('td')

        s = tds[0].get_text().split('.')
      
        if s[0][-3:] == 'Uhr':
            date = datetime.datetime.today().date()
        else:
            if len(s[2]) == 2:
                s[2] = "20"+s[2]
            
            date = datetime.datetime(int(s[2]),int(s[1]),int(s[0])).date()
            
        
        Category = tds[1].get_text().split(' ')[1]
    
        Analyst = tds[2].get_text()
        
        #print date, Category, Analyst
                  
        detailedPage = requests.get(url)
        soupNew = BeautifulSoup(detailedPage.content,'html.parser')
    
        string= soupNew.findAll("div",{"class":" teaser teaser-xs color-news"})[0].get_text()
        #print soupNew.findAll("p")
        dic = get_phrase(string)
        if dic is None:
            continue
        
        
        X = X.append({'Date':date,'Category':Category,'Analyst':Analyst,
                      'Phrase':dic['Phrase'],'TargetPrize':dic['TargetPrize'],
                     'FormerTargetPrize':dic['FormerTargetPrize']},ignore_index=True)
        
        
    #X.drop_duplicates(inplace=True)
    #X.reset_index(drop=True,inplace=True)
print X
        


Empty DataFrame
Columns: [Date, TargetPrize, FormerTargetPrize, Category, Analyst, Phrase]
Index: []
[20]
[23]
[27]
[23]
[21]
[18]
[19]
[17]
[18]
[19]
[20]
[19]
[19]
[28]
[15]
[20]
[20]
[21]
[15]
[18]
[25]
[14]
[21, 55]
[15]
[20]
[32]
[22]
[20]
[20]
[20]
[20]
[17]
[22]
[25]
[23]
[24]
[22]
[24]
[26]
          Date  TargetPrize  FormerTargetPrize        Category  \
0   2018-02-06        190.0              190.0             buy   
1   2018-02-05        203.0              203.0      overweight   
2   2018-02-05        205.0              205.0      Outperform   
3   2018-02-02        170.0              195.0  market-perform   
4   2018-02-02        162.0              199.0          Market   
5   2018-02-02        170.0              163.0            Hold   
6   2018-02-02        200.0              190.0          kaufen   
7   2018-02-02        170.0              174.0           Equal   
8   2018-02-02        210.0              200.0             buy   
9   2018-02-02        190.0             

In [315]:

X.drop_duplicates()
string = string.replace(",",".")
ListOfStrings = string.split(" ")

print ListOfStrings
print [i for i,x in enumerate(ListOfStrings) if x in ['US-Dollar','USD']]

[u'Der', u'Analyst', u'Barclays', u'Capital', u'hat', u'die', u'Einstufung', u'f\xfcr', u'3M', u'Co.', u'auf', u'"Underweight"', u'mit', u'einem', u'Kursziel', u'von', u'235', u'USD', u'belassen.']
[17]


In [310]:
url = "https://www.finanzen.net/analyse/3M_Underweight-Barclays_Capital_636363"
url = "https://www.finanzen.net/analyse/Volkswagen_VW_vz_buy-Deutsche_Bank_AG_638969"
detailedPage = requests.get(url)
soupNew = BeautifulSoup(detailedPage.content,'html.parser')

In [311]:
l = soupNew.findAll("div",{"class":" teaser teaser-xs color-news"})
print l[0].get_text()

FRANKFURT (dpa-AFX Analyser) - Die Deutsche Bank hat die Einstufung für Volkswagen nach Zahlen auf "Buy" belassen. Die Kostenkontrolle laufe beim Autobauer weiterhin gut, schrieb Analyst Tim Rokossa in einer am Donnerstag vorliegenden Studie. Er lobte zudem die erreichte Wende bei der Kernmarke VW. /tav/zbDatum der Analyse: 15.03.2018 Hinweis: Informationen zur Offenlegungspflicht bei Interessenkonflikten im Sinne von § 34 b WpHG für das genannte Analysten-Haus finden Sie unter http://web.dpa-afx.de/offenlegungspflicht/offenlegungs_pflicht.html.


In [323]:

l = soupNew.findAll("div",{"class":"box"})
print l[2]
x= l[2].findAll("tr")

string = x[0].get_text()
print string.split(':')



<div class="box"><h2 class="box-headline">Zusammenfassung: 3M reduce</h2><div class="spacer-10"></div><table class="table"><colgroup><col><col width="33%"><col width="33%"></col></col></col></colgroup><tr><td><strong>Unternehmen:</strong><br>3M Co.</br></td><td><strong>Analyst:</strong><br>Nomura</br></td><td><strong>Kursziel:</strong><br>88,00 USD</br></td></tr><tr><td><strong>Rating jetzt:</strong><br>reduce</br></td><td><strong>Kurs*:</strong><br>91,28 USD</br></td><td><strong>Abst. Kursziel*:</strong><br>-</br></td></tr><tr><td><strong>Rating vorher:</strong><br>neutral</br></td><td><strong>Kurs aktuell:</strong><br>234,29 USD</br></td><td><strong>Abst. Kursziel aktuell:</strong><br>-</br></td></tr><tr><td><strong>Analyst Name:</strong><br>-</br></td><td><strong>KGV*:</strong><br>14,3</br></td><td><a href="/Kursziele/3M">Ø Kursziel</a>:<br/>240,60 USD</td></tr></table><div class="spacer-5"></div><div class="medium-font light-grey">*zum Zeitpunkt der Analyse</div></div>
[u'Unternehm